<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов
# Задачи
Разработка модели анонимизации персональных данных
# Описание проекта
Необходимо защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы. Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [8]:
data.duplicated().sum()

153

In [11]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']
features.shape

(5000, 4)

<div class="alert alert-block alert-info">
<b>Вывод:</b>  датасет состоит из пяти столбцов и 5000 строк. Пропусков в данных нет. Дубликаты есть, но это может быть связано с особенностью датасета, поэтому оставим их. Столбец 'Страховые выплаты' - это целевой признак, остальные столбцы - признаки. Перед нами стоит задача закодировать персональные данные о клиентах (численные признаки) и предесказать будут ли страховые выплаты, то есть перед нами задача классификации.
</div>

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)
 a. Изменится. Приведите примеры матриц.
 b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

**Ответ:** Не изменится

**Обоснование:** 
$$
X' = X*P
$$
$$
w' = (X'^T X')^{-1} X'^T y = 
$$
$$
= ((X*P)^T (X*P))^{-1} (X*P)^T y = ((P)^T*(X)^T X*P)^{-1} (P)^T X^T y =
$$
$$
= P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y = P^{-1} (X^T X)^{-1} X^T y
$$
$$
Получаем
$$
$$
w' = P^{-1} w
$$
$$
a = Xw
$$
$$
a' = X'w' = XPP^{-1}w = Xw
$$
$$
a = a' = Xw
$$


## Алгоритм преобразования

In [18]:
features # матрица исходных признаков X

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
...,...,...,...,...
4995,0,28.0,35700.0,2
4996,0,34.0,52400.0,1
4997,0,20.0,33900.0,2
4998,1,22.0,32700.0,3


In [16]:
P = np.random.rand(4, 4) #Создаем случайную невырожденную матрицу
np.linalg.inv(P) # Проверяем на невырожденность, в случае, если обратной матрицы к P не существует, будет ошибка
X = features.dot(P) # Создаем матрицу X'
X

,0,1,2,3
0,38552.367550,47210.318194,41850.191119,37519.031589
1,29548.933291,36182.456136,32074.297762,28756.902440
2,16332.792964,19998.941848,17727.851568,15895.032812
3,32400.125369,39677.568380,35173.626675,31531.477327
4,20292.284715,24848.957293,22027.247398,19748.494046
...,...,...,...,...
4995,27747.096024,33978.076589,30121.124300,27003.260096
4996,40720.437612,49865.787066,44204.259493,39628.775346
4997,26342.278406,32258.653765,28597.049440,25636.059370
4998,25412.320319,31120.113257,27588.398383,24731.193580


In [24]:
X_back= round(X.dot(np.linalg.inv(P))) # декодируем признаки, умножив закодированную матрицу на обратную к P
X_back

,0,1,2,3
0,1.0,41.0,49600.0,1.0
1,0.0,46.0,38000.0,1.0
2,0.0,29.0,21000.0,-0.0
3,0.0,21.0,41700.0,2.0
4,1.0,28.0,26100.0,-0.0
...,...,...,...,...
4995,0.0,28.0,35700.0,2.0
4996,0.0,34.0,52400.0,1.0
4997,0.0,20.0,33900.0,2.0
4998,1.0,22.0,32700.0,3.0


**Алгоритм**


Cоздаем случайную квадратичную матрицу P, проверяем, чтобы она была невырожденной. Затем умножаем матрицу признаков на эту невырожденную матрицу, и получаем закодированные признаки. После обучаем модель и сравниваем качество моделей с зашифрованными признаками и нет.

In [ ]:
# создадим класс Линейной регрессии, и проверим точность предсказания с оригинальными и зашифрованными признаками
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = ((np.linalg.inv((X.T.dot(X)))).dot(X.T)).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

**Обоснование**

Также можем декодировать зашифрованные признаки, просто умножив измененную матрицу на матрицу обратную к P

## Проверка алгоритма

In [39]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
model_X = LinearRegression()
model_X.fit(X, target)
predictions_X = model_X.predict(X)
print('r2 метрика предсказания для оригинальных признаков', r2_score(target, predictions))
print('r2 метрика предсказания для зашифрованных признаков', r2_score(target, predictions_X))

r2 метрика предсказания для оригинальных признаков 0.42494550286668
r2 метрика предсказания для зашифрованных признаков 0.4249455028665382


Можем заметить, что метрики качества идентичны. (Различия в числах связаны в погрешностях машинных вычислений )

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные загружены
- [ ]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [ ]  Указан правильный вариант ответа
    - [ ]  Вариант обоснован
- [ ]  Выполнен шаг 3: предложен алгоритм преобразования
    - [ ]  Алгоритм описан
    - [ ]  Алгоритм обоснован
- [ ]  Выполнен шаг 4: алгоритм проверен
    - [ ]  Алгоритм реализован
    - [ ]  Проведено сравнение качества моделей до и после преобразования